In [26]:
import pandas as pd
from lifelines import CoxPHFitter
import numpy as np
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Read the Excel file into a pandas DataFrame
data = pd.read_excel('DATA1.xlsx')

# Standardize the covariates
scaler = StandardScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data.drop(['Months', 'DEATH'], axis=1)), columns=data.columns[2:])

# Add the 'Months' and 'DEATH' columns back to the scaled data
data_scaled[['Months', 'DEATH']] = data[['Months', 'DEATH']]

# Create a CoxPHFitter object
cph = CoxPHFitter()

# Fit the Cox proportional hazards model with the standardized covariates
cph.fit(data_scaled, duration_col='Months', event_col='DEATH')

# Print the summary of the model
#cph.print_summary()
print(cph.summary)

                            coef  exp(coef)  se(coef)  coef lower 95%  \
covariate                                                               
AGE                     0.216087   1.241211  0.088512        0.042608   
SEX                    -0.021036   0.979184  0.076603       -0.171175   
CompositeStage          0.886563   2.426774  0.091961        0.706322   
LNInvolment            -0.389643   0.677299  0.082473       -0.551287   
Comorbidity            -0.043602   0.957335  0.081518       -0.203374   
FamiliyHistoryOfCancer -0.091921   0.912177  0.078631       -0.246035   

                        coef upper 95%  exp(coef) lower 95%  \
covariate                                                     
AGE                           0.389567             1.043528   
SEX                           0.129104             0.842674   
CompositeStage                1.066804             2.026525   
LNInvolment                  -0.227998             0.576208   
Comorbidity                   0.11617

In [27]:
concordance_values = {}
for column in data_scaled.columns:
    try:
        concordance_value = concordance_index(data_scaled[column], -cph.predict_partial_hazard(data_scaled), data_scaled['DEATH'])
        concordance_values[column] = concordance_value
    except ZeroDivisionError:
        concordance_values[column] = None

# Print the concordance values
for column, value in concordance_values.items():
    print("Concordance value for", column, ":", value)

Concordance value for DEATH : None
Concordance value for AGE : 0.5394435640077478
Concordance value for SEX : 0.7171007184305057
Concordance value for CompositeStage : 0.2753710776966591
Concordance value for LNInvolment : 0.780604368533421
Concordance value for Comorbidity : 0.7227260734780028
Concordance value for FamiliyHistoryOfCancer : 0.7396111129865307
Concordance value for Months : 0.7210922981622961


In [28]:
print("Catagorical data:\n")
data_scaled['Months'] = data_scaled['Months'].astype('category')
data_scaled['DEATH'] = data_scaled['DEATH'].astype('category')
data_scaled['AGE'] = data_scaled['AGE'].astype('category')
data_scaled['SEX'] = data_scaled['SEX'].astype('category')
data_scaled['CompositeStage'] = data_scaled['CompositeStage'].astype('category')
data_scaled['LNInvolment'] = data_scaled['LNInvolment'].astype('category')
data_scaled['Comorbidity'] = data_scaled['Comorbidity'].astype('category')
data_scaled['FamiliyHistoryOfCancer'] = data_scaled['FamiliyHistoryOfCancer'].astype('category')
print(data_scaled.head())

Catagorical data:

  DEATH       AGE       SEX CompositeStage LNInvolment Comorbidity  \
0     0 -0.588591 -1.233717       0.032170    1.604031    0.913359   
1     0 -0.588591  0.810558      -2.174702   -0.623429    0.913359   
2     0 -0.422086 -1.233717      -1.071266   -0.623429    0.913359   
3     1 -0.172330  0.810558      -1.071266   -0.623429   -1.094860   
4     0  0.993201  0.810558       0.032170    1.604031    0.913359   

  FamiliyHistoryOfCancer Months  
0              -0.383611     70  
1              -0.383611     68  
2              -0.383611     69  
3              -0.383611     43  
4              -0.383611     71  


In [29]:
# Iterate over each column in the DataFrame
univariate_results = []
univariate_aic_bic = []
for column in data_scaled.columns:
    if data_scaled[column].dtype.name == 'category':
        for col in data.columns:
                if col not in ['Months', 'ID']:
                 cph_univariate = CoxPHFitter(penalizer=0.1)
                 cph_univariate.fit(data[['Months', 'ID', col]], duration_col='Months', event_col='ID', show_progress=True)
                 univariate_results.append((col, cph_univariate.print_summary()))

Iteration 1: norm_delta = 0.77722, step_size = 0.9500, log_lik = -1663.17959, newton_decrement = 70.49318, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.01332, step_size = 0.9500, log_lik = -1595.37158, newton_decrement = 0.02032, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00069, step_size = 0.9500, log_lik = -1595.35129, newton_decrement = 0.00005, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1595.35124, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.


<lifelines.CoxPHFitter: fitted with 343 total observations, 0 right-censored observations>
             duration col = 'Months'
                event col = 'ID'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 343
number of events observed = 343
   partial log-likelihood = -1595.35
         time fit was run = 2023-08-21 06:36:16 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
DEATH       1.50       4.49       0.13             1.24             1.76                 3.46                 5.82

            cmp to     z      p   -log2(p)
covariate                                 
DEATH         0.00 11.35 <0.005      96.84
---
Concordance = 0.66
Partial AIC = 3192.70
log-likelihood ratio test = 135.66 on 1 df
-log2(p) of ll-ratio test = 101.73

Iteration 1: norm_delta = 0.01879, step_size = 0.9500, log_lik = -1663.17959, newton_decrement = 0.06380, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00085, step_size = 0.9500, log_lik = -1663.11614, newton_decrement = 0.00013, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00004, step_size = 0.9500, log_lik = -1663.11600, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1663.11600, newton_decrement = 0.00000, seconds_since_start = 0.1
Convergence success after 4 iterations.


<lifelines.CoxPHFitter: fitted with 343 total observations, 0 right-censored observations>
             duration col = 'Months'
                event col = 'ID'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 343
number of events observed = 343
   partial log-likelihood = -1663.12
         time fit was run = 2023-08-21 06:36:16 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
AGE        -0.00       1.00       0.00            -0.01             0.01                 0.99                 1.01

            cmp to     z    p   -log2(p)
covariate                               
AGE           0.00 -0.36 0.72       0.47
---
Concordance = 0.51
Partial AIC = 3328.23
log-likelihood ratio test = 0.13 on 1 df
-log2(p) of ll-ratio test = 0.47

Iteration 1: norm_delta = 0.01792, step_size = 0.9500, log_lik = -1663.17959, newton_decrement = 0.06049, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00095, step_size = 0.9500, log_lik = -1663.11915, newton_decrement = 0.00017, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00005, step_size = 0.9500, log_lik = -1663.11898, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1663.11898, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.


...

<lifelines.CoxPHFitter: fitted with 343 total observations, 0 right-censored observations>
             duration col = 'Months'
                event col = 'ID'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 343
number of events observed = 343
   partial log-likelihood = -1659.20
         time fit was run = 2023-08-21 06:36:23 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
LNInvolment  -0.32       0.73       0.11            -0.54            -0.09                 0.58                 0.91

              cmp to     z    p   -log2(p)
covariate                                 
LNInvolment     0.00 -2.77 0.01       7.48
---
Concordance = 0.56
Partial AIC = 3320.40
log-likelihood ratio test = 7.96 on 1 df
-log2(p) of ll-ratio test = 7.71

Iteration 1: norm_delta = 0.06577, step_size = 0.9500, log_lik = -1663.17959, newton_decrement = 0.79658, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00275, step_size = 0.9500, log_lik = -1662.38897, newton_decrement = 0.00141, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00014, step_size = 0.9500, log_lik = -1662.38756, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1662.38756, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.


<lifelines.CoxPHFitter: fitted with 343 total observations, 0 right-censored observations>
             duration col = 'Months'
                event col = 'ID'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 343
number of events observed = 343
   partial log-likelihood = -1662.39
         time fit was run = 2023-08-21 06:36:23 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
Comorbidity  -0.13       0.88       0.10            -0.33             0.07                 0.72                 1.08

              cmp to     z    p   -log2(p)
covariate                                 
Comorbidity     0.00 -1.26 0.21       2.27
---
Concordance = 0.53
Partial AIC = 3326.78
log-likelihood ratio test = 1.58 on 1 df
-log2(p) of ll-ratio test = 2.26

Iteration 1: norm_delta = 0.02937, step_size = 0.9500, log_lik = -1663.17959, newton_decrement = 0.15086, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00064, step_size = 0.9500, log_lik = -1663.03168, newton_decrement = 0.00008, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00003, step_size = 0.9500, log_lik = -1663.03161, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1663.03161, newton_decrement = 0.00000, seconds_since_start = 0.0
Convergence success after 4 iterations.


<lifelines.CoxPHFitter: fitted with 343 total observations, 0 right-censored observations>
             duration col = 'Months'
                event col = 'ID'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 343
number of events observed = 343
   partial log-likelihood = -1663.03
         time fit was run = 2023-08-21 06:36:23 UTC

---
                         coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                      
FamiliyHistoryOfCancer   0.09       1.09       0.16            -0.22             0.39                 0.80                 1.48

                         cmp to    z    p   -log2(p)
covariate                                           
FamiliyHistoryOfCancer     0.00 0.55 0.58       0.78
---
Concordance = 0.50
Partial AIC = 3328.06
log-likelihood ratio test = 0.30 on 1 df
-log2(p) of ll-ratio test = 0.77

In [36]:
univariate_aic_bic = []
for column in data.columns:
    if column not in ['Months', 'ID']:
        cph_univariate = CoxPHFitter()
        cph_univariate.fit(data[['Months', 'ID', column]], duration_col='Months', event_col='ID', show_progress=True)
        log_likelihood = cph_univariate.log_likelihood_
        num_params = cph_univariate.params_.shape[0]
        n = data.shape[0]
        aic = -2 * log_likelihood + 2 * num_params
        bic = -2 * log_likelihood + num_params * np.log(n)
        univariate_aic_bic.append((column, aic, bic))

# Print the AIC and BIC values for each column
for column, aic, bic in univariate_aic_bic:
    print(f"Column: {column}\n")
    print(f"AIC: {aic}")
    print(f"BIC: {bic}\n")

Iteration 1: norm_delta = 0.91112, step_size = 0.9500, log_lik = -1663.17959, newton_decrement = 82.63783, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.02267, step_size = 0.9500, log_lik = -1583.96297, newton_decrement = 0.04759, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00124, step_size = 0.9500, log_lik = -1583.91536, newton_decrement = 0.00014, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -1583.91522, newton_decrement = 0.00000, seconds_since_start = 0.1
Convergence success after 4 iterations.
Iteration 1: norm_delta = 0.02076, step_size = 0.9500, log_lik = -1663.17959, newton_decrement = 0.07049, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00091, step_size = 0.9500, log_lik = -1663.10952, newton_decrement = 0.00014, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00005, step_size = 0.9500, log_lik = -1663.10939, newton_decrement = 0.00000, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, 

In [31]:
# Calculate p-values for each variable
p_values = []
summaries = []
for col in data.columns:
    if col not in ['Months', 'DEATH']:
        cph_univariate = CoxPHFitter(penalizer=0.1)
        cph_univariate.fit(data[['Months', 'DEATH', col]], duration_col='Months', event_col='DEATH', show_progress=True)
        p_values.append((col, cph_univariate.summary['p'][col]))
        summaries.append((col, cph_univariate.summary))

# Sort the p-values list in ascending order
p_values.sort(key=lambda x: x[1])

# Get the significant variable with the lowest p-value
significant_variable_pvalue = p_values[0][0]
significant_variable_pvalue_value = data[significant_variable_pvalue].iloc[0]

# Find the summary of the significant variable
significant_variable_summary = None
for summary in summaries:
    if summary[0] == significant_variable_pvalue:
        significant_variable_summary = summary[1]
        break

# Print the summary of the significant variable's operations
if significant_variable_summary is not None:
    print(significant_variable_summary)

print(f"\nSignificant variable based on p-value: {significant_variable_pvalue}")
#print(f"Value of the significant variable: {significant_variable_pvalue_value}")


Iteration 1: norm_delta = 0.35071, step_size = 0.9500, log_lik = -943.70062, newton_decrement = 15.41644, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.01406, step_size = 0.9500, log_lik = -928.49246, newton_decrement = 0.02456, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00073, step_size = 0.9500, log_lik = -928.46793, newton_decrement = 0.00007, seconds_since_start = 0.1
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -928.46787, newton_decrement = 0.00000, seconds_since_start = 0.1
Convergence success after 4 iterations.
Iteration 1: norm_delta = 0.09094, step_size = 0.9500, log_lik = -943.70062, newton_decrement = 0.84604, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.00646, step_size = 0.9500, log_lik = -942.84679, newton_decrement = 0.00411, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00033, step_size = 0.9500, log_lik = -942.84269, newton_decrement = 0.00001, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_si

In [32]:
print("Catagorical data:\n")
data[significant_variable_pvalue] = data[significant_variable_pvalue].astype('category')

# Print the categorical variable
print(f"\nCategorical variable: {data[significant_variable_pvalue]}")

# Get the summary of the categorical variable
categorical_variable_summary = data[significant_variable_pvalue].describe()

# Print the summary of the categorical variable
print(f"\nSummary of the categorical variable:\n{categorical_variable_summary}")

Catagorical data:


Categorical variable: 0      3
1      1
2      2
3      2
4      3
      ..
338    3
339    2
340    2
341    4
342    4
Name: CompositeStage, Length: 343, dtype: category
Categories (4, int64): [1, 2, 3, 4]

Summary of the categorical variable:
count     343
unique      4
top         4
freq      123
Name: CompositeStage, dtype: int64


In [33]:
cph_multivariate = CoxPHFitter(penalizer=0.1)
cph_multivariate.fit(data[['Months', 'DEATH', significant_variable_pvalue]], duration_col='Months', event_col='DEATH', show_progress=True)
multivariate_summary = cph_multivariate.summary

# Print the summary of the multivariate analysis
print(f"\nMultivariate analysis summary:\n{multivariate_summary}")

Iteration 1: norm_delta = 0.60207, step_size = 0.9500, log_lik = -943.70062, newton_decrement = 33.86694, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.06882, step_size = 0.9500, log_lik = -909.00329, newton_decrement = 0.38483, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00442, step_size = 0.9500, log_lik = -908.61639, newton_decrement = 0.00154, seconds_since_start = 0.0
Iteration 4: norm_delta = 0.00000, step_size = 1.0000, log_lik = -908.61485, newton_decrement = 0.00000, seconds_since_start = 0.1
Convergence success after 4 iterations.

Multivariate analysis summary:
                    coef  exp(coef)  se(coef)  coef lower 95%  coef upper 95%  \
covariate                                                                       
CompositeStage  0.707114   2.028129  0.087683        0.535259        0.878968   

                exp(coef) lower 95%  exp(coef) upper 95%  cmp to         z  \
covariate                                                                    
Com

In [35]:
log_likelihood = cph_multivariate.log_likelihood_

# Calculate the number of parameters
num_parameters = len(cph_multivariate.params_)

# Calculate the AIC value
n = len(data)  # number of observations
aic_value = -2 * log_likelihood + 2 * num_parameters

# Calculate the BIC value
bic_value = -2 * log_likelihood + np.log(n) * num_parameters

# Get the column names
column_names = [significant_variable_pvalue]

# Print the AIC and BIC values with column names
print(f"Column Names: {column_names}")
print(f"AIC value: {aic_value}")
print(f"BIC value: {bic_value}")

Column Names: ['CompositeStage']
AIC value: 1819.2296914692981
BIC value: 1823.0674219164641


In [37]:
c_index = cph_multivariate.concordance_index_

# Get the column names
column_names = [significant_variable_pvalue]

# Print the column names and concordance index
print(f"Column Names: {column_names}")
print(f"Concordance Index: {c_index}")

Column Names: ['CompositeStage']
Concordance Index: 0.6820927111294652
